# JUICE RPWI HF SID3 (Full): L1a QL -- 2024/7/20

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import datetime
import glob

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_sid3_lib as juice_sid3
import juice_math_lib as juice_math

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 1                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump
#
time_mode = 0                           # 0: Epoch    1: data number

# *** Parameter set ***
cal = 2                                 # 0: background     1: CAL
p_raw_max = 9.5                         # background: 7.5   CAL: 10
p_raw_min = 2.0                         # background: 2.5   CAL: 5

# *** Frenquency range set ***
f_mode_min = 800                        # 80
f_mode_max = 40000                      # 2068 : as same as SID-4/20

# *** Conversion factor: cal_mode ***
cf, p_max0, p_min0, str_unit = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0, "   str_unit:", str_unit)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

In [ ]:
# *** Ground - Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name_list = ['SID03_20231024-0036.cdf']
data_name_list = ['SID03-2ch-comp0-20231117-1438.cdf']
data_name_list = ['SID03-2ch-comp1-20231117-1448.cdf']
data_name_list = ['SID03-2ch-comp2-20231117-1500.cdf']
data_name_list = ['SID03-3ch-comp0-20231117-1424.cdf']
data_name_list = ['SID03-3ch-comp1-20231117-1418.cdf']                      # 
"""
data_name_list = ['SID03-3ch-comp2-20231117-1429.cdf']
data_name_list = ['SID03-3ch-comp3-20231117-1432.cdf']
"""

"""
# 202310 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'        # CDF data folder
data_name_list = ['SID03_Seq03.cdf']
data_name_list = ['SID03_Seq04.cdf']
data_name_list = ['SID03_Seq10.cdf']
data_name_list = ['SID03_Seq11.cdf']
data_name_list = ['SID03_Seq12.cdf']
data_name_list = ['SID03_Seq13.cdf']
data_name_list = ['SID03_Seq14.cdf']
data_name_list = ['SID03_Seq15.cdf']
data_name_list = ['SID03_Seq17.cdf']
data_name_list = ['SID03_Seq03.cdf',
                  'SID03_Seq04.cdf',
                  'SID03_Seq10.cdf',
                  'SID03_Seq11.cdf',
                  'SID03_Seq12.cdf',
                  'SID03_Seq13.cdf',
                  'SID03_Seq15.cdf',
                  'SID03_Seq17.cdf',
                  'SID03_Seq14.cdf',
                 ]

# 202401 -- ErrorCheck
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202401_FS/cdf/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_comp2.cdf',
                 ]

# 202405 -- dryrun check
# TEST
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/1_first_parts/'
# COMP1->2, 2->1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T040602-20000108T041954_V01___TMIDX_00000.bin.cdf',  # pc2_rpwi_hf_SID3_without_masking_2024_05_15_14_02_53
                 ]
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T042923-20000108T044315_V01___TMIDX_00000.bin.cdf',  # pc2_rpwi_hf_SID3_with_masking_2024_05_15_14_26_24
                 ]
# COMP2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T050103-20000108T051303_V01___TMIDX_00000.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-1-Stack_RPWI_2_2024_05_15_14_57_41/TM00
                  'JUICE_L1a_RPWI-HF-SID3_20000108T051318-20000108T051533_V01___TMIDX_00001.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-1-Stack_RPWI_2_2024_05_15_14_57_41/TM01
                 ]
# COMP
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T051649-20000108T052219_V01___TMIDX_00000.bin.cdf',  # pc2_RPWI_combined-Stack_RPWI_3_2024_05_15_15_13_36
                 ]
#
# LGA: COMP-1
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/2_LGA-dryrun/'   
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T052829-20000108T053159_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-1_2024_05_15_15_25_20
                  'JUICE_L1a_RPWI-HF-SID3_20000108T054433-20000108T055503_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM00
                  'JUICE_L1a_RPWI-HF-SID3_20000108T055533-20000108T060533_V01___TMIDX_00001.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM01
                  'JUICE_L1a_RPWI-HF-SID3_20000108T060603-20000108T060703_V01___TMIDX_00002.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13a-2_2024_05_15_15_41_26/TM02
                  'JUICE_L1a_RPWI-HF-SID3_20000108T064409-20000108T064839_V01___TMIDX_00000.bin.cdf',  # pc2_LGA-dryrun_RPW-SEQ-13b_2024_05_15_16_40_53
                 ]
# EGA: COMP-1 & -2
data_dir = '/Users/user/0-python/JUICE_data/CCSDS_test_v2/test_TMIDX/2405_dryrun_test/3_EGA-dryrun/'   
# COMP-2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T065600-20000108T065800_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04a_2024_05_15_16_52_23
                  'JUICE_L1a_RPWI-HF-SID3_20000108T065858-20000108T070058_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04b-1_2024_05_15_16_55_43
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070151-20000108T070351_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04c_2024_05_15_16_58_39
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070445-20000108T070645_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04b-2_2024_05_15_17_01_40
                  'JUICE_L1a_RPWI-HF-SID3_20000108T070750-20000108T070950_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-04d_2024_05_15_17_04_35
                 ]
# COMP-1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T071058-20000108T071258_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12a-1_2024_05_15_17_07_46
                  'JUICE_L1a_RPWI-HF-SID3_20000108T071707-20000108T071907_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12b_2024_05_15_17_13_55
                  'JUICE_L1a_RPWI-HF-SID3_20000108T071951-20000108T072151_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12a-2_2024_05_15_17_16_42
                  'JUICE_L1a_RPWI-HF-SID3_20000108T072241-20000108T072441_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12c_2024_05_15_17_19_28
                  'JUICE_L1a_RPWI-HF-SID3_20000108T072530-20000108T072730_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12d_2024_05_15_17_22_18
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073057-20000108T073257_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12e_2024_05_15_17_27_45
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073349-20000108T073549_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12f_2024_05_15_17_30_37
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073635-20000108T073835_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12g_2024_05_15_17_33_23
                  'JUICE_L1a_RPWI-HF-SID3_20000108T073920-20000108T074120_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12h_2024_05_15_17_36_08
                 ]
# COMP-2
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T074210-20000108T074410_V01___TMIDX_00000.bin.cdf',  # pc2_SC_roll_RPW-SEQ-12i-2_2024_05_15_17_38_55
                 ]
# COMP-1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000108T074511-20000108T074711_V01___TMIDX_00000.bin.cdf',  # pc2_EGA-dryrun_RPW-SEQ-12j_2024_05_15_17_41_59
                 ]

# 202406-7 -- dryrun check
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202406_EM3/cdf/'
# COMP1->2, 2->1
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240614T161148-20240614T162539_V01___TMIDX_00000.bin.cdf',  
                 ]

# 202407 -- LEGA check
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240715/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240716T152808-20240716T153933_V01.cdf',
                 ]
"""
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240718/'
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20000101T011258-20000101T013701_V01___TMIDX_00000.bin.cdf',
                 ]


In [ ]:
# *** Flight data: Ver.2 ***
"""
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/01/26/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240126T083835_V01.cdf',
                 ]

# 20240701
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/07/01/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID3_20240701T011127_V01.cdf',
                 ]
"""

"""
# 20240706
data_dir = '/Users/user/Dropbox-Univ/data/data-JUICE/datasets/2024/07/06/'        # CDF data folder
data_name_list = [# 'JUICE_L1a_RPWI-HF-SID3_20240706T121009_V01.cdf',
                  'JUICE_L1a_RPWI-HF-SID3_20240706T175202_V01.cdf',
                 ]
"""

In [ ]:
# *** Flight - Ver. 1 ***
"""
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/'        # CDF data folder

# *** 2023/4/20 before deployment of MAG-BOOM
data_name_list = ['04/20/SID3_20230420T103558-20230420T103800.cdf']

# *** 2023/5/23 before deployment of RWI
data_name_list = ['05/23/SID3_20230523T090033-20230523T091021.cdf',
                  '05/23/SID3_20230523T092629-20230523T093617.cdf',
                  '05/23/SID3_20230523T095226-20230523T100214.cdf',
                  '05/23/SID3_20230523T102036-20230523T102247.cdf',
                 ]
# *** 2023/5/23 after X deployment of RWI
data_name_list = ['05/23/SID3_20230523T102431-20230523T102647.cdf',
                  '05/23/SID3_20230523T104536-20230523T104747.cdf',
                 ]
# *** 2023/5/23 after Z deployment of RWI
data_name_list = ['05/23/SID3_20230523T104931-20230523T105148.cdf',
                  '05/23/SID3_20230523T110537-20230523T110749.cdf',
                 ]
# *** 2023/5/23 after Y deployment of RWI
data_name_list = ['05/23/SID3_20230523T110933-20230523T111149.cdf']

# *** 2023/5/23 background
data_name_list = ['05/23/SID3_20230523T121538-20230523T122526.cdf',
                  '05/23/SID3_20230523T124134-20230523T125122.cdf',
                  '05/23/SID3_20230523T125738-20230523T130726.cdf',
                 ]
"""

"""
# *** 20230525 
data_name_list = ['05/25/SID3_20230525T130149-20230525T131137.cdf',
                  '05/25/SID3_20230525T150435-20230525T151423.cdf',
                  '05/25/SID3_20230525T165313-20230525T170301.cdf',
                 ]

# *** ALL ***
data_name_list = ['04/20/SID3_20230420T103558-20230420T103800.cdf',
                  '05/23/SID3_20230523T090033-20230523T091021.cdf',
                  '05/23/SID3_20230523T092629-20230523T093617.cdf',
                  '05/23/SID3_20230523T095226-20230523T100214.cdf',
                  '05/23/SID3_20230523T102036-20230523T102247.cdf',
                  '05/23/SID3_20230523T102431-20230523T102647.cdf',
                  '05/23/SID3_20230523T104536-20230523T104747.cdf',
                  '05/23/SID3_20230523T104931-20230523T105148.cdf',
                  '05/23/SID3_20230523T110537-20230523T110749.cdf',
                  '05/23/SID3_20230523T110933-20230523T111149.cdf',
                  '05/23/SID3_20230523T121538-20230523T122526.cdf',
                  '05/23/SID3_20230523T124134-20230523T125122.cdf',
                  '05/23/SID3_20230523T125738-20230523T130726.cdf',
                  '05/25/SID3_20230525T130149-20230525T131137.cdf',
                  '05/25/SID3_20230525T150435-20230525T151423.cdf',
                  '05/25/SID3_20230525T165313-20230525T170301.cdf',
                 ]
"""

In [ ]:
# *** Prelaunch - Ver.1 ***
"""
# *** 202105
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202105/'        # CDF data folder
data_name_list = ['SID3_20210531_SCPFM_PTR_RPWI_2_day4_xid32813.cdf']
data_name_list = ['SID3_20210531_SCPFM_PTR_RPWI_2_day5_xid32814.cdf']

# *** 202106 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202106/'        # CDF data folder
data_name_list = ['SID3_SCTBTV_Phase3_xid32776.cdf']
data_name_list = ['SID3_SCTBTV_Phase5_xid32794.cdf']
data_name_list = ['SID3_SCTBTV_Phase11_xid32836.cdf']
data_name_list = ['SID3_SCTBTV_Phase13_xid32848.cdf']
                  
# *** 202111 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202111/'        # CDF data folder
data_name_list = ['SID3_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32816.cdf',
                  'SID3_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32817.cdf',
                  'SID3_SCPFM_PTR_RPWI_delta.RPWI_SCM_TEST_xid32818.cdf',
                 ]

# *** 202204 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202204/'        # CDF data folder
data_name_list = ['SID3_20220400_SCPFM_EMCR.Phase4_xid32994.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid32995.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid32996.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid32997.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid32998.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid32999.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid33000.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid33001.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid33002.cdf',
                  'SID3_20220400_SCPFM_EMCR.Phase4_xid33003.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32798.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32799.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32800.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32801.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32802.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32803.cdf',
                  'SID3_20220400_SCPFM_EMCR.RPWI_NCR_xid32804.cdf',
                 ] 
                
# *** 202211 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202211/'        # CDF data folder
data_name_list = ['SID3_20221115_Mission_Test_GCO500_1RPR_4.cdf',
                  'SID3_20221115_Mission_Test_GCO500_1RPR_6.cdf',
                  ]

# *** 202301 ***
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/prelaunch/202301/'        # CDF data folder
data_name_list = ['SID3_20230113_Mission_Test_CSW3.2.1_SCOP020_1RPR_4.cdf',
                  'SID3_20230113_Mission_Test_CSW3.2.1_SCOP020_1RPR_6.cdf',
                  ]
"""

In [ ]:
"""
# *** Group read
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202407_FS/cdf_20240716/'        # CDF data folder
data_name = '*SID3*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid3.hf_sid3_read(cdf)

    if i==0:
        data = data1
        print(data.EuEu.shape)
    else:
        data = juice_sid3.hf_sid3_add(data, data1)
        print(data.EuEu.shape)    

data_name = os.path.split(data_name)[1];  print(data_name)

In [ ]:
date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_time0 = data.EuEu.shape[0]
n_freq0 = data.EuEu.shape[1]
print("data size:", data.EuEu.shape, n_time0, n_freq0)
# print(data.epoch)

In [ ]:
# Mode 
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
if (data.complex[0] == 0):
    if N_ch == 3:
        print("complex:", data.complex[0], " (power 3-ch) <Table 6.2.2>")
    else:
        print("complex:", data.complex[0], " (power 2-ch) <Table 6.2.3>")
    N_component = N_ch
elif (data.complex[0] == 1):
    if N_ch == 3:
        print("complex:", data.complex[0], " (nominal spectral matrix 3-ch) <Table 6.2.4>")
    else:
        print("complex:", data.complex[0], " (nominal spectral matrix 2-ch) <Table 6.2.5>")
    N_component = N_ch * N_ch
elif (data.complex[0] == 2):
    if N_ch == 3:
        N_component = 27 + 3
        print("complex:", data.complex[0], " (Polarization separation 3-ch) <Table 6.2.6>")
    else:
        N_component = 12 + 3
        print("complex:", data.complex[0], " (Polarization separation 2-ch) <Table 6.2.7>")
else:
    print("complex:", data.complex[0], " (3D spectral matrix 3-ch) <Table 6.2.8>")
    N_component = 21
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("BG_downlink:", data.BG_downlink[0])
print("RFI_rejection:", data.RFI_rejection[0])
N_frequency = data.frequency.shape[1]
print("==> Component", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time0)

# Data for checks

In [ ]:
Epoch_1d       = data.epoch.tolist()
freq_1d        = data.frequency[0].tolist()
freq1          = freq_1d[0:N_frequency]
freq_width_1d  = data.freq_width[0].tolist()
freq_width1    = freq_width_1d[0:N_frequency]

EuEu_2d = data.EuEu.transpose()
EvEv_2d = data.EvEv.transpose()
EwEw_2d = data.EwEw.transpose()
if data.complex[0] > 0:    # Matrix
    E_DoLuv_2d = data.E_DoLuv.transpose()
    E_DoLvw_2d = data.E_DoLvw.transpose()
    E_DoLwu_2d = data.E_DoLwu.transpose()
    E_DoCuv_2d = data.E_DoCuv.transpose()
    E_DoCvw_2d = data.E_DoCvw.transpose()
    E_DoCwu_2d = data.E_DoCwu.transpose()
    E_ANGuv_2d = data.E_ANGuv.transpose()
    E_ANGvw_2d = data.E_ANGvw.transpose()
    E_ANGwu_2d = data.E_ANGwu.transpose()
freq_width_2d  = data.freq_width.transpose()
if band_mode == 1:
    EuEu_2d = EuEu_2d / freq_width_2d / 1000
    EvEv_2d = EvEv_2d / freq_width_2d / 1000
    EwEw_2d = EwEw_2d / freq_width_2d / 1000
    """
    print("frequency width:", freq_width1)
    """
#
f_min0 = freq_1d[0]
f_max0 = freq_1d[N_frequency-1]
power_str = juice_cdf.power_label(band_mode, unit_mode)
print(power_str, "   f_MAX/min:", f_max0, f_min0)

# print(freq1); print(freq_width1)

# Raw

In [ ]:
if data.complex[0] == 0:    # power
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.add_subplot(4, 1, 1);  ax7 = fig.add_subplot(4, 1, 2); ax8 = fig.add_subplot(4, 1, 3);  ax9 = fig.add_subplot(4, 1, 4)
else:                       # matrix
    fig = plt.figure(figsize=(12, 16))
    ax1 = fig.add_subplot(9, 1, 1); ax7 = fig.add_subplot(9, 1, 7);  ax8 = fig.add_subplot(9, 1, 8);  ax9 = fig.add_subplot(9, 1, 9)
    ax2 = fig.add_subplot(9, 1, 2); ax3 = fig.add_subplot(9, 1, 3);  ax4 = fig.add_subplot(9, 1, 4)
    ax5 = fig.add_subplot(9, 1, 5); ax6 = fig.add_subplot(9, 1, 6) 

ax1.plot(np.ravel(data.EuEu), '-r', linewidth=1.2, label='uu')
ax1.plot(np.ravel(data.EvEv), '-g', linewidth=0.8, label='vv')
ax1.plot(np.ravel(data.EwEw), '-b', linewidth=0.5, label='ww')
ax1.plot(np.ravel(data.BG_Eu), ':r', linewidth=1.2, label='BG_uu')
ax1.plot(np.ravel(data.BG_Ev), ':g', linewidth=0.8, label='BG_vv')
ax1.plot(np.ravel(data.BG_Ew), ':b', linewidth=0.5, label='BG_ww')
ax1.set_yscale('log')
ax7.plot(np.ravel(data.frequency),     '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*100),  '-g', linewidth=0.8, label='step*100')
ax7.plot(np.ravel(data.freq_width*100), '-b', linewidth=1.0, label='width*100')
ax8.plot(np.ravel(data.T_HF_FPGA),  ':r', label='T (HK-FPGA)')
ax8.plot(np.ravel(data.T_RWI_CH1),  ':b', label='T (RWI1)')
ax8.plot(np.ravel(data.T_RWI_CH2),  ':g', label='T (RWI2)')
ax8.plot(np.ravel(data.cal_signal*10), '-k', label='CAL')
ax8.plot(np.ravel(data.onboard_cal*15), '-r', label='Onboard-CAL')
ax8.plot(np.ravel(data.RFI_rejection*10+2), '_b', label='RFI')
ax8.plot(np.ravel(data.complex*10+4), '_r', label='comp')
ax9.plot(np.ravel(data.epoch), '.')
#
if data.complex[0] > 0:    # Matrix
    ax2.plot(np.ravel(data.EuEv_re), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(data.EvEw_re), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(data.EwEu_re), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(data.EuEv_im), ':r', linewidth=1.2,  label='uv_im')
    ax2.plot(np.ravel(data.EvEw_im), ':g', linewidth=1.0,  label='vw_im')
    ax2.plot(np.ravel(data.EwEu_im), ':b', linewidth=0.8,  label='wu_im')
    #
    ax3.plot(np.ravel(data.E_DoPuv), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(data.E_DoPvw), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(data.E_DoPwu), '-b', linewidth=0.6, label='wu')
    #
    ax4.plot(np.ravel(data.E_DoLuv), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(data.E_DoLvw), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(data.E_DoLwu), '-b', linewidth=0.6, label='wu')
    #
    ax5.plot(np.ravel(data.E_DoCuv), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(data.E_DoCvw), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(data.E_DoCwu), '-b', linewidth=0.6, label='wu')
    #
    ax6.plot(np.ravel(data.E_ANGuv), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(data.E_ANGvw), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(data.E_ANGwu), '-b', linewidth=0.6, label='wu')

if data.complex[0] == 2:    # Matrix
    ax1.plot(np.ravel(data.EuEu_NC), '-r', linewidth=1.2, label='uu')
    ax1.plot(np.ravel(data.EvEv_NC), '-g', linewidth=0.8, label='vv')
    ax1.plot(np.ravel(data.EwEw_NC), '-b', linewidth=0.5, label='ww')
    ax1.plot(np.ravel(data.EuEu_RC), '.r', linewidth=1.2, label='uu')
    ax1.plot(np.ravel(data.EvEv_RC), '.g', linewidth=0.8, label='vv')
    ax1.plot(np.ravel(data.EwEw_RC), '.b', linewidth=0.5, label='ww')
    ax1.plot(np.ravel(data.EuEu_LC), '_r', linewidth=1.2, label='uu')
    ax1.plot(np.ravel(data.EvEv_LC), '_g', linewidth=0.8, label='vv')
    ax1.plot(np.ravel(data.EwEw_LC), '_b', linewidth=0.5, label='ww')
  
    ax2.plot(np.ravel(data.EuEv_re_NC), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(data.EvEw_re_NC), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(data.EwEu_re_NC), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(data.EuEv_im_NC), ':r', linewidth=1.2,  label='uv_im')
    ax2.plot(np.ravel(data.EvEw_im_NC), ':g', linewidth=1.0,  label='vw_im')
    ax2.plot(np.ravel(data.EwEu_im_NC), ':b', linewidth=0.8,  label='wu_im')
    #
    ax3.plot(np.ravel(data.E_DoPuv_NC), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(data.E_DoPvw_NC), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(data.E_DoPwu_NC), '-b', linewidth=0.6, label='wu')
    #
    ax4.plot(np.ravel(data.E_DoLuv_NC), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(data.E_DoLvw_NC), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(data.E_DoLwu_NC), '-b', linewidth=0.6, label='wu')
    #
    ax5.plot(np.ravel(data.E_DoCuv_NC), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(data.E_DoCvw_NC), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(data.E_DoCwu_NC), '-b', linewidth=0.6, label='wu')
    #
    ax6.plot(np.ravel(data.E_ANGuv_NC), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(data.E_ANGvw_NC), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(data.E_ANGwu_NC), '-b', linewidth=0.6, label='wu')

ax1.set_ylabel(power_str);  ax8.set_ylabel('T [degC]');   ax9.set_ylabel('Date-Time')
ax8.set_xlabel(str_date)
if data.complex[0] > 0:    # Matrix
    ax2.set_ylabel('Real & Img');          ax3.set_ylabel('Deg. Pol.');    ax4.set_ylabel('Deg. Linear Pol.')
    ax5.set_ylabel('Deg. Circular Pol.');  ax6.set_ylabel('Linear Deg.');  ax7.set_ylabel('frequency [kHz]')
#
title_label = '[JUICE/RPWI HF Full (SID-3)]  ' + data_name
ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax7.legend(loc='upper right', fontsize=8); ax8.legend(loc='upper right', fontsize=8)
if data.complex[0] > 0:    # Matrix
    ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
    ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8); 

# range: X-axis
"""
num_x = 10
xlim=[0, N_frequency*num_x-1]
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim); ax7.set_xlim(xlim)
xlim=[0, num_x-1]; ax8.set_xlim(xlim)
"""
# range: Y-axis
if data.complex[0] > 0:    # Matrix
    if N_ch < 3:
        ylim=[-10**(p_raw_max-3), 10**(p_raw_max-3)];  ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min0, f_max0]
ax7.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
# print(data.epoch)
# print(data.T_HF_FPGA, data.T_RWI_CH1, data.T_RWI_CH2)
# print(freq1)
print(data.complex)
print(data.cal_signal)
print(data.onboard_cal)
print(data.RFI_rejection)
print(data.Pol_sep_SW)

In [ ]:
# Sweep_num
n0 = 0;   n1 = n_time0//2;  n2 = n_time0-1  # 11

# Peak
print(f"[First - {n0}]")
peak_E = np.ravel(data.EuEu[n0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EvEv[n0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EwEw[n0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
print(f"[Mid - {n1}]")
peak_E = np.ravel(data.EuEu[n1]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EvEv[n1]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EwEw[n1]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
print(f"[Last - {n2}]")
peak_E = np.ravel(data.EuEu[n2]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EvEv[n2]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = np.ravel(data.EwEw[n2]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")

In [ ]:
n1 = n_time0-1;  
print(f"[SWEEP - {n1}]")

peak_E = np.ravel(data.EuEu[n1]); peak_f = np.argmax(peak_E); 
print("Peak at", '{:.1f}'.format(freq1[peak_f]), "kHz")

print("   EuEu:", '{:+.2e}'.format(np.ravel(data.EuEu[n1])[peak_f]),    "     EvEv:", '{:+.2e}'.format(np.ravel(data.EvEv[n1])[peak_f]),    "     EwEw:", '{:+.2e}'.format(np.ravel(data.EwEw[n1])[peak_f]))
print("EuEv_re:", '{:+.2e}'.format(np.ravel(data.EuEv_re[n1])[peak_f]), "  EvEw_re:", '{:+.2e}'.format(np.ravel(data.EvEw_re[n1])[peak_f]), "  EwEu_re:", '{:+.2e}'.format(np.ravel(data.EwEu_re[n1])[peak_f]))
print("EuEv_im:", '{:+.2e}'.format(np.ravel(data.EuEv_im[n1])[peak_f]), "  EvEw_im:", '{:+.2e}'.format(np.ravel(data.EvEw_im[n1])[peak_f]), "  EwEu_im:", '{:+.2e}'.format(np.ravel(data.EwEu_im[n1])[peak_f]))
print("")
print("  DoPuv:", '{:+.2e}'.format(np.ravel(data.E_DoPuv[n1])[peak_f]), "    DoLuv:", '{:+.2e}'.format(np.ravel(data.E_DoLuv[n1])[peak_f]), "    DoCuv:", '{:+.2e}'.format(np.ravel(data.E_DoCuv[n1])[peak_f]), "    ANGuv:", '{:+.2e}'.format(np.ravel(data.E_ANGuv[n1])[peak_f]))
print("  DoPvw:", '{:+.2e}'.format(np.ravel(data.E_DoPvw[n1])[peak_f]), "    DoLvw:", '{:+.2e}'.format(np.ravel(data.E_DoLvw[n1])[peak_f]), "    DoCvw:", '{:+.2e}'.format(np.ravel(data.E_DoCvw[n1])[peak_f]), "    ANGvw:", '{:+.2e}'.format(np.ravel(data.E_ANGvw[n1])[peak_f]))
print("  DoPwu:", '{:+.2e}'.format(np.ravel(data.E_DoPwu[n1])[peak_f]), "    DoLwu:", '{:+.2e}'.format(np.ravel(data.E_DoLwu[n1])[peak_f]), "    DoCwu:", '{:+.2e}'.format(np.ravel(data.E_DoCwu[n1])[peak_f]), "    ANGwu:", '{:+.2e}'.format(np.ravel(data.E_ANGwu[n1])[peak_f]))

# UV
print('Stockes-UV: {:+.2e}'.format(np.ravel(data.E_Iuv[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Quv[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Uuv[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Vuv[n1])[peak_f]))
print('Stockes-VW: {:+.2e}'.format(np.ravel(data.E_Ivw[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Qvw[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Uvw[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Vvw[n1])[peak_f]))
print('Stockes-WU: {:+.2e}'.format(np.ravel(data.E_Iwu[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Qwu[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Uwu[n1])[peak_f]),   '{:+.2e}'.format(np.ravel(data.E_Vwu[n1])[peak_f]))

## Spec

In [ ]:
# Sweep_num
n0 = 0;   n1 = n_time0//2;  n2 = n_time0-1  # 11
# Y-range
p_min = p_min0;  p_max = p_max0-2
# X-range
f_min = f_min0;  f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max

if data.complex[0] == 0:    # power
    fig = plt.figure(figsize=(12, 4))
    ax1 = fig.add_subplot(1, 1, 1)
else:                      # matrix
    fig = plt.figure(figsize=(12, 12))
    ax1 = fig.add_subplot(4, 1, 1);  ax5 = fig.add_subplot(4, 1, 2);  ax6 = fig.add_subplot(4, 1, 3);  ax7 = fig.add_subplot(4, 1, 4)
    # ax3 = fig.add_subplot(6, 1, 2);  ax4 = fig.add_subplot(6, 1, 3)
 
ax1.plot(freq1, data.EuEu[n0], ':r', linewidth=1.0, label='uu init')
ax1.plot(freq1, data.EvEv[n0], ':g', linewidth=0.8, label='vv init')
ax1.plot(freq1, data.EwEw[n0], ':b', linewidth=0.6, label='ww init')
ax1.plot(freq1, data.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, data.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, data.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, data.EuEu[n2], '-r', linewidth=1.0, label='uu end')
ax1.plot(freq1, data.EvEv[n2], '-g', linewidth=0.8, label='vv end')
ax1.plot(freq1, data.EwEw[n2], '-b', linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
#
if data.complex[0] > 0:    # Matrix
    """
    ax3.plot(freq1, data.EuEv_re[n0], ':r', linewidth=1.0, label='uv_re half')
    ax3.plot(freq1, data.EvEw_re[n0], ':g', linewidth=0.8, label='vw_re half')
    ax3.plot(freq1, data.EwEu_re[n0], ':b', linewidth=0.6, label='wu_re half')
    ax3.plot(freq1, data.EuEv_re[n1], '-.r', linewidth=1.0, label='uv_re half')
    ax3.plot(freq1, data.EvEw_re[n1], '-.g', linewidth=0.8, label='vw_re half')
    ax3.plot(freq1, data.EwEu_re[n1], '-.b', linewidth=0.6, label='wu_re half')
    ax3.plot(freq1, data.EuEv_im[n1], '-r', linewidth=1.2,  label='uv_im half')
    ax3.plot(freq1, data.EvEw_im[n1], '-g', linewidth=1.0,  label='vw_im half')
    ax3.plot(freq1, data.EwEu_im[n1], '-b', linewidth=0.8,  label='wu_im half')
    #
    ax4.plot(freq1, data.E_DoPuv[n0], ':r', linewidth=1.0, label='uv init')
    ax4.plot(freq1, data.E_DoPvw[n0], ':g', linewidth=0.8, label='vw init')
    ax4.plot(freq1, data.E_DoPwu[n0], ':b', linewidth=0.6, label='wu init')
    ax4.plot(freq1, data.E_DoPuv[n1], '-.r', linewidth=1.2,  label='uv half')
    ax4.plot(freq1, data.E_DoPvw[n1], '-.g', linewidth=1.0,  label='vw half')
    ax4.plot(freq1, data.E_DoPwu[n1], '-.b', linewidth=0.8,  label='wu half')
    ax4.plot(freq1, data.E_DoPuv[n2], '-r', linewidth=1.2,  label='uv end')
    ax4.plot(freq1, data.E_DoPvw[n2], '-g', linewidth=1.0,  label='vw end')
    ax4.plot(freq1, data.E_DoPwu[n2], '-b', linewidth=0.8,  label='wu end')
    """
    #
    ax5.plot(freq1, data.E_DoLuv[n0], ':r', linewidth=1.0, label='uv init')
    ax5.plot(freq1, data.E_DoLvw[n0], ':g', linewidth=0.8, label='vw init')
    ax5.plot(freq1, data.E_DoLwu[n0], ':b', linewidth=0.6, label='wu init')
    ax5.plot(freq1, data.E_DoLuv[n1], '-.r', linewidth=1.2,  label='uv half')
    ax5.plot(freq1, data.E_DoLvw[n1], '-.g', linewidth=1.0,  label='vw half')
    ax5.plot(freq1, data.E_DoLwu[n1], '-.b', linewidth=0.8,  label='wu half')
    ax5.plot(freq1, data.E_DoLuv[n2], '-r', linewidth=1.2,  label='uv end')
    ax5.plot(freq1, data.E_DoLvw[n2], '-g', linewidth=1.0,  label='vw end')
    ax5.plot(freq1, data.E_DoLwu[n2], '-b', linewidth=0.8,  label='wu end')
    #
    ax6.plot(freq1, data.E_DoCuv[n0], ':r', linewidth=1.0, label='uv init')
    ax6.plot(freq1, data.E_DoCvw[n0], ':g', linewidth=0.8, label='vw init')
    ax6.plot(freq1, data.E_DoCwu[n0], ':b', linewidth=0.6, label='wu init')
    ax6.plot(freq1, data.E_DoCuv[n1], '-.r', linewidth=1.2,  label='uv half')
    ax6.plot(freq1, data.E_DoCvw[n1], '-.g', linewidth=1.0,  label='vw half')
    ax6.plot(freq1, data.E_DoCwu[n1], '-.b', linewidth=0.8,  label='wu half')
    ax6.plot(freq1, data.E_DoCuv[n2], '-r', linewidth=1.2,  label='uv end')
    ax6.plot(freq1, data.E_DoCvw[n2], '-g', linewidth=1.0,  label='vw end')
    ax6.plot(freq1, data.E_DoCwu[n2], '-b', linewidth=0.8,  label='wu end')
    #
    ax7.plot(freq1, data.E_ANGuv[n0], ':r', linewidth=1.0, label='uv init')
    ax7.plot(freq1, data.E_ANGvw[n0], ':g', linewidth=0.8, label='vw init')
    ax7.plot(freq1, data.E_ANGwu[n0], ':b', linewidth=0.6, label='wu init')
    ax7.plot(freq1, data.E_ANGuv[n1], '-.r', linewidth=1.2,  label='uv half')
    ax7.plot(freq1, data.E_ANGvw[n1], '-.g', linewidth=1.0,  label='vw half')
    ax7.plot(freq1, data.E_ANGwu[n1], '-.b', linewidth=0.8,  label='wu half')
    ax7.plot(freq1, data.E_ANGuv[n2], '-r', linewidth=1.2,  label='uv end')
    ax7.plot(freq1, data.E_ANGvw[n2], '-g', linewidth=1.0,  label='vw end')
    ax7.plot(freq1, data.E_ANGwu[n2], '-b', linewidth=0.8,  label='wu end')
    #
    if f_mode == 1:
        ax5.set_xscale('log');  ax6.set_xscale('log');  ax7.set_xscale('log')   # ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel(power_str);  ax7.set_xlabel('frequency [Hz]')
if data.complex[0] > 0:    # Matrix
    ax5.set_ylabel('Deg. Linear Pol.');  ax6.set_ylabel('Deg. Circular Pol.');  ax7.set_ylabel('Linear Deg.')
    ax4.set_ylabel('Deg. Pol.');         ax3.set_ylabel('Real & Img')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)
if data.complex[0] > 0:    # Matrix
    ax3.legend(loc='upper right', fontsize=8); ax4.legend(loc='upper right', fontsize=8); 
    ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8);  ax7.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min, f_max]
ax1.set_xlim(xlim); 
if data.complex[0] > 0:    # Matrix
    ax5.set_xlim(xlim); ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); 
# range: Y-axis
ylim=[10**p_min, 10**p_max]; ax1.set_ylim(ylim)
if data.complex[0] > 0:    # Matrix
    ylim=[-0.05, 1.05]; ax5.set_ylim(ylim);  ax4.set_ylim(ylim); 
    ylim=[-1.05, 1.05]; ax6.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax7.set_ylim(ylim)
    if N_ch < 3:
        ylim=[-10**(p_raw_max-3), 10**(p_raw_max-3)];  ax3.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

## FT

In [ ]:
# Y-range
f_min = f_min0;      f_max = f_max0
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
# Z-range
p_min = p_min0+2.5;    p_max = p_max0-3

# Epoch_1d
num_1d = np.arange(n_time0)

fig2d = plt.figure(figsize=[12,14])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3)
if time_mode == 1:
    ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylim(f_min, f_max);        ax2.set_ylim(f_min, f_max);        ax3.set_ylim(f_min, f_max)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EuEu');  ax2.set_title('EvEv'); ax3.set_title('EwEw')
if time_mode == 1:
    ax4.set_xlabel(str_date)
else:
    ax3.set_xlabel(str_date)

# Plot
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(power_str)

# X-axis: Time
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    E_min = '2024-01-26 08:40:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    E_max = '2024-01-26 09:40:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
    xlim=[t_min, t_max]
    xlim=[Epoch_1d[0], Epoch_1d[-1]]
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

## Ang-T

In [ ]:
if data.complex[0] > 0:    # Matrix
    # Y-range
    f_min0 = f_min;      f_max0 = f_max
    if f_mode == 0:
        f_min = f_mode_min;  f_max = f_mode_max
    # Z-range
    p_min = p_min0;      p_max = p_max0

    fig2d = plt.figure(figsize=[12,33])
    ax1 = fig2d.add_subplot(10, 1, 1);  ax2 = fig2d.add_subplot(10, 1, 2);  ax3 = fig2d.add_subplot(10, 1, 3)
    ax4 = fig2d.add_subplot(10, 1, 4);  ax5 = fig2d.add_subplot(10, 1, 5);  ax6 = fig2d.add_subplot(10, 1, 6)
    ax7 = fig2d.add_subplot(10, 1, 7);  ax8 = fig2d.add_subplot(10, 1, 8);  ax9 = fig2d.add_subplot(10, 1, 9)
    if time_mode == 1:
        ax10 = fig2d.add_subplot(10, 1, 10)

    # Y-axis
    ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
    ax4.set_ylim(f_min0, f_max0);      ax5.set_ylim(f_min0, f_max0);      ax6.set_ylim(f_min0, f_max0)
    ax7.set_ylim(f_min0, f_max0);      ax8.set_ylim(f_min0, f_max0);      ax9.set_ylim(f_min0, f_max0)
    if f_mode == 1:
        ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
        ax4.set_yscale('log');         ax5.set_yscale('log');             ax6.set_yscale('log')
        ax7.set_yscale('log');         ax8.set_yscale('log');             ax9.set_yscale('log')
    ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')
    ax4.set_ylabel('Frequency [kHz]'); ax5.set_ylabel('Frequency [kHz]'); ax6.set_ylabel('Frequency [kHz]')
    ax7.set_ylabel('Frequency [kHz]'); ax8.set_ylabel('Frequency [kHz]'); ax9.set_ylabel('Frequency [kHz]')

    # X-axis
    ax1.set_title("[" + data_name + "]  " + 'Deg. Lin. uv');     ax2.set_title('Deg. Lin. vw');     ax3.set_title('Deg. Lin. wu')
    ax4.set_title('Deg. Cir. uv');     ax5.set_title('Deg. Cir. vw');     ax6.set_title('Deg. Cir. wu')
    ax7.set_title('Ang. uv');          ax8.set_title('Ang. vw');          ax9.set_title('Ang. wu')
    if time_mode == 1:
        ax10.set_xlabel(str_date)
    else:
        ax9.set_xlabel(str_date)

    # Plot
    if time_mode == 1:
        p1 = ax1.pcolormesh(num_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p2 = ax2.pcolormesh(num_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p3 = ax3.pcolormesh(num_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p4 = ax4.pcolormesh(num_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(num_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(num_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(num_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
        p8 = ax8.pcolormesh(num_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
        p9 = ax9.pcolormesh(num_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
        p10 = ax10.plot(np.ravel(data.epoch[:]), '.')
        pp9 = fig2d.colorbar(p9, ax=ax10, orientation="vertical"); pp9.set_label(power_str)
    else:
        p1 = ax1.pcolormesh(Epoch_1d, freq_1d, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p2 = ax2.pcolormesh(Epoch_1d, freq_1d, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p3 = ax3.pcolormesh(Epoch_1d, freq_1d, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
        p4 = ax4.pcolormesh(Epoch_1d, freq_1d, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p5 = ax5.pcolormesh(Epoch_1d, freq_1d, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p6 = ax6.pcolormesh(Epoch_1d, freq_1d, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
        p7 = ax7.pcolormesh(Epoch_1d, freq_1d, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
        p8 = ax8.pcolormesh(Epoch_1d, freq_1d, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
        p9 = ax9.pcolormesh(Epoch_1d, freq_1d, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
    pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)
    pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(power_str)
    pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(power_str)
    pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical");  pp1.set_label(power_str)
    pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical");  pp2.set_label(power_str)
    pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical");  pp3.set_label(power_str)
    pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical");  pp1.set_label(power_str)
    pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical");  pp2.set_label(power_str)
    pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical");  pp3.set_label(power_str)

    # X-axis
    if time_mode == 1:
        xlim=[num_1d[0], num_1d[-1]]
        ax10.set_xlim(xlim)
    else:
        E_min = '2024-01-26 08:40:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-01-26 09:40:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
        xlim=[t_min, t_max]
        xlim=[Epoch_1d[0], Epoch_1d[-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim)
    ax6.set_xlim(xlim); ax7.set_xlim(xlim); ax8.set_xlim(xlim); ax9.set_xlim(xlim)

    plt.show()
    if dump_mode == 1:
        png_fname = work_dir+data_name+'_AngT.png'
        if f_mode == 1:
            png_fname = work_dir+data_name+'_AngT-log.png'
        fig2d.savefig(png_fname)

# CLEANED data

In [ ]:
EuEu_c = juice_math.clean_rfi(np.ravel(data.EuEu))
EvEv_c = juice_math.clean_rfi(np.ravel(data.EvEv))
EwEw_c = juice_math.clean_rfi(np.ravel(data.EwEw))
EuEu_c = EuEu_c.reshape(n_time0, n_freq0)
EvEv_c = EvEv_c.reshape(n_time0, n_freq0)
EwEw_c = EwEw_c.reshape(n_time0, n_freq0)
EuEu_c_2d = EuEu_c.transpose()
EvEv_c_2d = EvEv_c.transpose()
EwEw_c_2d = EwEw_c.transpose()

## CLEANED - Spec

In [ ]:
# Sweep_num
n0 = 0;  n1 = np.int16(n_time0/2);  n2 = n_time0-1
# X-range
f_min0 = f_min;  f_max0 = f_max
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
# Y-range
p_min = p_min0;  p_max = p_max0

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

ax1.plot(freq1, data.EuEu[n0], ':r', linewidth=1.0, label='uu init')
ax1.plot(freq1, data.EvEv[n0], ':g', linewidth=0.8, label='vv init')
ax1.plot(freq1, data.EwEw[n0], ':b', linewidth=0.6, label='ww init')
ax1.plot(freq1, data.EuEu[n1], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, data.EvEv[n1], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, data.EwEw[n1], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, data.EuEu[n2], '-r', linewidth=1.0, label='uu end')
ax1.plot(freq1, data.EvEv[n2], '-g', linewidth=0.8, label='vv end')
ax1.plot(freq1, data.EwEw[n2], '-b', linewidth=0.6, label='ww end')
ax2.plot(freq1, EuEu_c[n0], ':r', linewidth=1.0, label='uu init cleaned')
ax2.plot(freq1, EvEv_c[n0], ':g', linewidth=0.8, label='vv init cleaned')
ax2.plot(freq1, EwEw_c[n0], ':b', linewidth=0.6, label='ww init cleaned')
ax2.plot(freq1, EuEu_c[n1], '-.r', linewidth=1.0, label='uu half cleaned')
ax2.plot(freq1, EvEv_c[n1], '-.g', linewidth=0.8, label='vv half cleaned')
ax2.plot(freq1, EwEw_c[n1], '-.b', linewidth=0.6, label='ww half cleaned')
ax2.plot(freq1, EuEu_c[n2], '-r', linewidth=1.0, label='uu end cleaned')
ax2.plot(freq1, EvEv_c[n2], '-g', linewidth=0.8, label='vv end cleaned')
ax2.plot(freq1, EwEw_c[n2], '-b', linewidth=0.6, label='ww end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log')

# Label
ax1.set_ylabel(power_str);  ax2.set_ylabel(power_str)
ax2.set_xlabel('frequency [Hz]')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3 
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min0, f_max0]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);

# Plot
fig.show
png_fname = work_dir+data_name+'_spec-c.png'
if f_mode == 1:
    png_fname = work_dir+data_name+'_spec-log-c.png'
    # fig.savefig(png_fname

## CLEANED -- FT

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
if f_mode == 0:
    f_min = f_mode_min;  f_max = f_mode_max
# Z-range
p_min = p_min0+2.5;  p_max = p_max0-3

fig2d = plt.figure(figsize=[12,14])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3)
if time_mode == 1:
    ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EuEu cleaned');     ax2.set_title('EvEv cleaned');     ax3.set_title('EwEw cleaned')
if time_mode == 1:
    ax4.set_xlabel(str_date)
else:
    ax3.set_xlabel(str_date)

# Plot
if time_mode == 1:
    p1 = ax1.pcolormesh(num_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p2 = ax2.pcolormesh(num_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p3 = ax3.pcolormesh(num_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min, vmax=10**p_max), cmap='jet')
    p4 = ax4.plot(np.ravel(data.epoch[:]), '.')
    pp3 = fig2d.colorbar(p3, ax=ax4, orientation="vertical"); pp3.set_label(power_str)
else:
    p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
    p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
    p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(power_str)

# X-axis
if time_mode == 1:
    xlim=[num_1d[0], num_1d[-1]]
    ax4.set_xlim(xlim)
else:
    E_min = '2024-01-26 08:40:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    E_max = '2024-01-26 09:40:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  
    xlim=[t_min, t_max]
    xlim=[Epoch_1d[0], Epoch_1d[-1]]
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim)

plt.show()
png_fname = work_dir+data_name+'_FT-c.png'
if f_mode == 1:
    png_fname = work_dir+data_name+'_FT-log-c.png'
    # fig2d.savefig(png_fname)